# Primary Care data

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8) 
options(tibble.width = Inf, width = 300)

In [ ]:
library(DBI)
library(dbplyr)
library(bigrquery)
suppressPackageStartupMessages(library(tidyverse))
library(lubridate)
library(readxl)
library(knitr)
library(IRdisplay)
library(htmltools)

In [ ]:
con <- DBI::dbConnect(
    bigquery(), 
    project = "yhcr-prd-bradfor-bia-core", 
    bigint = "character", 
    page_size = 5000,
    dataset = 'CB_2617'
)


In [ ]:
index_iuc_tbl <- tbl(con, "index_iuc")

In [ ]:
index_iuc_tbl %>% glimpse()

In [ ]:
gp_tbl <- tbl(con, "tbl_PrimaryCare_SRCode")

In [ ]:
#gp_tbl %>% glimpse()

In [ ]:
# Codes associated with consultations face-to-face or via telephone

In [ ]:
gp_codes_df <- read_csv('../data/richard_pilbery-primary-care-clinician-patient-interaction-72708505.csv')

In [ ]:
gp_codes_df %>% glimpse()

In [ ]:
#DBI::dbWriteTable(con, "gp_codes", gp_codes_df, overwrite = TRUE)

In [ ]:
gp_sql <- "
    SELECT
      iuc.*,
      pc.*
    FROM
      index_iuc AS iuc
    JOIN tbl_PrimaryCare_SRCode AS pc
        ON iuc.person_id = pc.person_id
    INNER JOIN gp_codes GPC
        ON LOWER(pc.CTV3Code) = LOWER(GPC.code)
    WHERE
      pc.DateEventRecorded BETWEEN iuc.index_call_date
                              AND DATE_ADD(iuc.index_call_date, INTERVAL 7 DAY)

"

In [ ]:
gp_df <- dbGetQuery(con, gp_sql)

In [ ]:
gp_df %>% glimpse()

### Note: Currently using DateEventRecorded since most of the start and end values are null.

In [ ]:
gp_df %>%
    mutate(
        floor_hour = lubridate::floor_date(DateEventRecorded, unit = 'day') # Hour might be more granular but does return same consultation across two hours.
    ) %>% count(person_id, floor_hour) %>% arrange(person_id, floor_hour)

In [ ]:
gp_df %>% filter(person_id == "PERSON_ID")

In [ ]:
gp_tbl %>% filter(person_id == "PERSON_ID") %>% glimpse()